In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict

In [2]:
movie_titles = pd.read_csv('movie_titles.csv', header=None, encoding='ISO-8859-1', 
                          names=['movie_id', 'year', 'title', 'description1', 'description2',
                                'description3'])

movie_titles = movie_titles.drop(columns=['description1', 'description2', 'description3'], axis=1)

In [3]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [4]:
PATH = '/Users/alecclarkfeather/capstone_project/'

train_hit_rate0 = pd.read_csv(PATH + 'hit_rate_train_test_folds/train_df_hit_rate0.csv')
test_hit_rate0 = pd.read_csv(PATH + 'hit_rate_train_test_folds/test_df_hit_rate0.csv')

In [5]:
#train_hit_rate0.head()

,Unnamed: 0,user_id,rating,date,movie_id,top_rated
0,653142,826574,4.0,2003-11-06,9729,0
1,4513338,2200645,4.0,2005-11-06,1719,0
2,3242458,2150434,5.0,2005-09-30,5356,0
3,3874119,2142065,3.0,2003-02-12,8753,0
4,2493566,867086,4.0,2005-04-24,6721,0


In [6]:
#test_hit_rate0.head()

,Unnamed: 0,user_id,rating,date,movie_id,top_rated
0,3407090,1667994,3.0,2004-05-08,7904,0
1,1201432,1105606,2.0,2004-07-05,10359,0
2,1313034,2129781,4.0,2004-12-06,8636,0
3,2478179,1129575,4.0,2005-08-22,2519,0
4,4818390,1001105,4.0,2004-03-16,8966,0


In [7]:
#train_hit_rate0['top_rated'].value_counts()

0    2333222
Name: top_rated, dtype: int64

In [8]:
train_hit_rate0 = train_hit_rate0.drop(columns=['Unnamed: 0', 'top_rated', 'date'], axis=1)
test_hit_rate0 = test_hit_rate0.drop(columns=['Unnamed: 0', 'top_rated', 'date'], axis=1)

In [9]:
train_hit_rate0 = train_hit_rate0[['user_id', 'movie_id', 'rating']]
test_hit_rate0 = test_hit_rate0[['user_id', 'movie_id', 'rating']]

In [10]:
#train_hit_rate0.head()

,user_id,movie_id,rating
0,826574,9729,4.0
1,2200645,1719,4.0
2,2150434,5356,5.0
3,2142065,8753,3.0
4,867086,6721,4.0


In [11]:
#test_hit_rate0.head()

,user_id,movie_id,rating
0,1667994,7904,3.0
1,1105606,10359,2.0
2,2129781,8636,4.0
3,1129575,2519,4.0
4,1001105,8966,4.0


In [12]:
#os.mkdir(PATH + 'hit_rate_folds_actual')

In [13]:
HIT_RATE_PATH_ACTUAL = PATH + 'hit_rate_folds_actual'

#train_hit_rate0.to_csv(HIT_RATE_PATH_ACTUAL + '/train_hr0.csv', index=False, header=None)
#test_hit_rate0.to_csv(HIT_RATE_PATH_ACTUAL + '/test_hr0.csv', index=False, header=None)

In [14]:
top_user_ratings_dict = pickle.load(open(PATH + 'top_user_ratings.pkl', 'rb'))

In [16]:
#top_user_ratings_dict[2129781]

10378

In [48]:
def get_top_n(predictions, n=100):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [49]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
test_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = NormalPredictor()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline1a = get_top_n(preds)
    

In [78]:
len(top_n_baseline1a['2129781'])

6

In [50]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline1a.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    recs_for_user = [int(rec[0]) for rec in top_n_baseline1a[user_id]]
    if top_movie_id in recs_for_user:
        recs_with_top_movie += 1

hit_rate1a = recs_with_top_movie / total_users
print(hit_rate1a)

0.0


In [51]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
train_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = NormalPredictor()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline1b = get_top_n(preds)

In [52]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline1b.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    recs_for_user = [int(rec[0]) for rec in top_n_baseline1b[user_id]]
    if top_movie_id in recs_for_user:
        recs_with_top_movie += 1

hit_rate1b = recs_with_top_movie / total_users
print(hit_rate1b)

0.0


In [53]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
test_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = KNNBaseline(sim_options=sim_options)

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline2a = get_top_n(preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [54]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline2a.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    recs_for_user = [int(rec[0]) for rec in top_n_baseline2a[user_id]]
    if top_movie_id in recs_for_user:
        recs_with_top_movie += 1

hit_rate2a = recs_with_top_movie / total_users
print(hit_rate2a)

0.0


In [55]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
train_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

sim_options = {"name": "pearson_baseline", "user_based": False}
baseline_algo = KNNBaseline(sim_options=sim_options)

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline2b = get_top_n(preds)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [56]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline2b.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    recs_for_user = [int(rec[0]) for rec in top_n_baseline2b[user_id]]
    if top_movie_id in recs_for_user:
        recs_with_top_movie += 1

hit_rate2b = recs_with_top_movie / total_users
print(hit_rate2b)

0.0


In [57]:
reader = Reader(sep=',',rating_scale=(1,5))

train_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
test_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = SVD()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline3a = get_top_n(preds)

In [58]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline3a.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    recs_for_user = [int(rec[0]) for rec in top_n_baseline3a[user_id]]
    if top_movie_id in recs_for_user:
        recs_with_top_movie += 1

hit_rate3a = recs_with_top_movie / total_users
print(hit_rate3a)

0.0


In [79]:
reader = Reader(sep=',',rating_scale=(1,5))

test_file = HIT_RATE_PATH_ACTUAL + '/train_hr0.csv'
train_file = HIT_RATE_PATH_ACTUAL + '/test_hr0.csv'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)

pkf = PredefinedKFold()

baseline_algo = SVD()

for trainset, testset in pkf.split(data):
    baseline_algo.fit(trainset)
    preds = baseline_algo.test(testset)
    
top_n_baseline3b = get_top_n(preds)

In [80]:
baseline_algo

In [86]:
top_user_ratings_df = pd.DataFrame.from_dict(top_user_ratings_dict, orient='index')
top_user_ratings_df.head()

,0
841872,14454
689085,299
2529854,11165
2597445,10886
2336656,3282


In [87]:
len(top_user_ratings_df)

264059

In [88]:
top_user_ratings_df = top_user_ratings_df.reset_index()
top_user_ratings_df.head()

,index,0
0,841872,14454
1,689085,299
2,2529854,11165
3,2597445,10886
4,2336656,3282


In [89]:
top_user_ratings_df.columns = ['user_id', 'movie_id']
top_user_ratings_df.head()

,user_id,movie_id
0,841872,14454
1,689085,299
2,2529854,11165
3,2597445,10886
4,2336656,3282


In [90]:
df_ratings = pd.read_csv('netflix_sample.csv')
df_ratings.head()

,Unnamed: 0,user_id,rating,date,movie_id
0,36420325,841872,3.0,2003-03-21,6464
1,42791681,689085,2.0,2004-05-19,7590
2,64859971,2529854,4.0,2005-10-22,11837
3,59155484,2597445,4.0,2005-01-25,10886
4,30218564,2336656,3.0,2005-07-01,5522


In [91]:
df_ratings = df_ratings.drop('Unnamed: 0', axis=1)
df_ratings.head()

,user_id,rating,date,movie_id
0,841872,3.0,2003-03-21,6464
1,689085,2.0,2004-05-19,7590
2,2529854,4.0,2005-10-22,11837
3,2597445,4.0,2005-01-25,10886
4,2336656,3.0,2005-07-01,5522


In [92]:
top_ratings_df = pd.merge(top_user_ratings_df, df_ratings, on=['user_id', 'movie_id'], how='left')
top_ratings_df.head()

,user_id,movie_id,rating,date
0,841872,14454,5.0,2004-04-25
1,689085,299,5.0,2003-12-02
2,2529854,11165,5.0,2005-10-22
3,2597445,10886,4.0,2005-01-25
4,2336656,3282,5.0,2005-06-11


In [93]:
top_user_ratings_df.head()

,user_id,movie_id
0,841872,14454
1,689085,299
2,2529854,11165
3,2597445,10886
4,2336656,3282


In [94]:
top_user_ratings_dict[2597445]

10886

In [95]:
top_ratings_df = top_ratings_df[['user_id', 'movie_id', 'rating']]
top_ratings_df.head()

,user_id,movie_id,rating
0,841872,14454,5.0
1,689085,299,5.0
2,2529854,11165,5.0
3,2597445,10886,4.0
4,2336656,3282,5.0


In [102]:
#preds_idk = []

pred_idk = baseline_algo.predict(str(top_ratings_df.loc[0]['user_id']),
                     str(top_ratings_df.loc[0]['movie_id']),
                     top_ratings_df.loc[0]['rating'])

In [106]:
pred_idk[3]

3.6324884644495894

In [101]:
top_n_baseline3b['841872']

[('1110', 4.162051742128112),
 ('14766', 3.8500097325128944),
 ('482', 3.8320850742477948),
 ('13651', 3.8071741374251546),
 ('1428', 3.7867858068428064),
 ('30', 3.7856255844736633),
 ('6037', 3.6776904450835737),
 ('12299', 3.6049011570635288),
 ('2015', 3.594508651448337),
 ('14254', 3.593241683130565),
 ('10621', 3.5725821717022455),
 ('2699', 3.400001833126414),
 ('14545', 3.3688461630852915),
 ('13342', 3.3638076728643638),
 ('4420', 3.209981818700854),
 ('4951', 3.1818164641556432),
 ('2112', 3.137173230056924),
 ('15555', 3.0809482131922463),
 ('11677', 3.0799486449830975),
 ('7991', 2.969385034387851),
 ('16242', 2.8911099924555947),
 ('13466', 2.6534115578855255)]

In [109]:
total_users = 0
recs_with_top_movie = 0

for user_id in top_n_baseline3b.keys():
    top_movie_id = top_user_ratings_dict[int(user_id)]
    total_users += 1
    num_recs = len(top_n_baseline3b[user_id])
    top_number = int(num_recs * 0.25)
    boundary_for_user = [rec[1] for rec in top_n_baseline3b[user_id]][top_number]
    pred_top_movie = baseline_algo.predict(user_id, 
                                           str(top_movie_id), 
                                           top_ratings_df[(top_ratings_df['user_id']==int(user_id)) &
                                                         (top_ratings_df['movie_id']==top_movie_id)]['rating'])
    if float(pred_top_movie[3]) > boundary_for_user:
        recs_with_top_movie += 1
    

hit_rate3b = recs_with_top_movie / total_users
print(hit_rate3b)

0.4611265265051608


In [110]:
top_ratings_df.to_csv(PATH + 'top_ratings_df.csv', index=False)

In [76]:
len([int(rec[0]) for rec in top_n_baseline3b['']])

14

In [ ]:
train_hr0 = pd.read_csv(HIT_RATE_PATH_ACTUAL + '/train_hr0.csv', header=None)
test_hr0 = pd.read_csv(HIT_RATE_PATH_ACTUAL + '/test_hr0.csv', header=None)

In [ ]:
train_hr0.columns = ['user_id', 'movie_id', 'rating']
test_hr0.columns = ['user_id', 'movie_id', 'rating']

In [ ]:
train_movie_ids = list(train_hr0['movie_id'].unique())
test_movie_ids = list(test_hr0['movie_id'].unique())

In [ ]:
train_user_ids = list(train_hr0['user_id'].unique())
test_user_ids = list(test_hr0['user_id'].unique())

In [ ]:
class DeepRecRmseModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        embedding_dimension = 16
        
        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_user_ids)+1, embedding_dimension)
        ])
        
        self.movie_embeddings = tf.keras.Sequential([
            tf.keras.layers.IntegerLookup(vocabulary=train_movie_ids, mask_token=None),
            tf.keras.layers.Embedding(len(train_movie_ids)+1, embedding_dimension)
        ])
        
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
    def call(self, inputs):
        user_id, movie_id = inputs
        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_id)
        return self.ratings(tf.concat([tf.reshape(user_embedding, shape=[-1,1]), 
                                       tf.reshape(movie_embedding, shape=[-1,1])], axis=1))

In [ ]:
class NetflixRmseModel(tfrs.models.Model):
    
    def __init__(self):
        super().__init__()
        self.rmse_model: tf.keras.Model = DeepRecRmseModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(),
                                                             metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.rmse_model((features['user_id'], features['movie_id']))
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        labels = features.pop('rating')
        predicted_ratings = self(features)
        return self.task(labels=labels, predictions=predicted_ratings)

In [ ]:
deep_baseline_model = NetflixRmseModel()
deep_baseline_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.3))

In [ ]:
train_hr0_tf = tf.data.Dataset.from_tensor_slices(dict(train_hr0))

for x in train_rmse0_tf.take(5):
    print(x)

In [ ]:
train_rmse0_tf = train_rmse0_tf.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']
})

In [ ]:
history_baseline = deep_baseline_model.fit(train_rmse0_tf, epochs=1, batch_size=20000)